In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile
import joblib

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import metrics
from sklearn.metrics import recall_score, precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
make_scorer = sklearn.metrics.make_scorer
f1 = make_scorer(f1_score, pos_label=1, average="binary")
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import RepeatedStratifiedKFold

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [2]:
## Build XGBoost Model
from copy import deepcopy
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix,roc_curve
import xgboost as xgb
import time
import warnings

# 1. Data Preparation

In [3]:
trainDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\0d1\\df_train_cbMSIn0d14nonInDI_STDnorm.csv")
diDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\0d1\\df_nonInDI_cbMSIn0d14nonInDI_STDnorm.csv")

In [4]:
trainDEFSDf

,pixel_id,311.15,269.25,435.2,301.25,241.1,738.55,857.55,867.55,299.1,267.1,794.55,337.2,339.1,233.0,328.15,124.0,279.1,type
0,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,3.953210,0.561634,0.526007,-0.160672,1.020894,-1.060231,-0.241555,-0.261111,2.281684,1.847785,-0.476776,2.573681,2.457422,0.755008,-0.975763,-0.102862,-0.843752,0
1,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,5.499377,1.488873,0.256846,-0.271471,1.328109,-0.871081,-0.168275,-0.261111,3.225803,2.603708,-0.476776,2.733761,3.349393,1.730237,-0.553603,1.203171,-0.628910,0
2,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,3.655719,0.621905,-0.137142,-0.057255,0.946003,-1.106408,-0.443515,-0.261111,1.937129,1.625724,-0.476776,1.777652,2.445633,2.006355,-1.057817,-0.969198,-0.628777,0
3,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,3.412890,0.428166,-0.086066,-0.275708,1.237134,-1.058673,-0.913550,-0.261111,1.543656,2.336013,1.612714,2.338674,2.351152,1.552337,-1.187617,-1.022450,-0.766370,0
4,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,2.092271,0.750785,0.624481,-0.051827,0.741049,-1.077604,-0.591638,-0.261111,-0.334251,1.129372,-0.476776,1.323991,1.454999,0.464660,-1.028013,-0.260324,-0.620623,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90955,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.273014,-1.042871,1.085864,-0.876582,-1.203295,-1.416813,-1.308037,-0.261111,-0.334251,-1.028769,-0.476776,-1.196404,0.406250,0.687848,-0.532158,0.910092,-0.896244,1
90956,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.096808,-1.042871,1.345307,-1.077784,0.675202,-0.614252,-1.469057,-0.261111,2.964883,-1.028769,-0.476776,-1.196404,0.488193,0.410449,-0.691905,-0.276832,-1.338888,1
90957,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.213884,1.253220,0.874884,-1.425507,-1.203295,-0.478005,-1.083904,-0.261111,2.279959,-1.028769,-0.476776,-1.196404,0.449698,1.274415,-0.836330,0.162720,-1.001561,1
90958,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.402346,-1.042871,0.667639,-1.401248,-1.203295,-0.380958,-0.753560,-0.261111,2.972262,-1.028769,-0.476776,0.462734,0.308840,0.654135,-0.776998,-0.247323,-0.993475,1


In [5]:
can = []
for i in list(range(0,5)) + list(range(6,10)) + list(range(11,12)) + list(range(14,16)) + [len(list(trainDEFSDf.columns))-1]:
    can.append(list(trainDEFSDf.columns)[i])

dican = []
for i in list(range(0,5)) + list(range(6,10)) + list(range(11,12)) + list(range(14,16)) + [len(list(diDEFSDf.columns))-1]:
    dican.append(list(diDEFSDf.columns)[i])

In [6]:
can

['pixel_id',
 '311.15',
 '269.25',
 '435.2',
 '301.25',
 '738.55',
 '857.55',
 '867.55',
 '299.1',
 '794.55',
 '233.0',
 '328.15',
 'type']

In [7]:
dican

['pixel_id',
 '311.15',
 '269.25',
 '435.2',
 '301.25',
 '738.55',
 '857.55',
 '867.55',
 '299.1',
 '794.55',
 '233.0',
 '328.15',
 'type']

In [8]:
## input training set ## 90960 x 20 df
trainDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\0d1\\df_train_cbMSIn0d14nonInDI_STDnorm.csv")
#trainDEFSDf[trainDEFSDf.type .== 1, :]
## calculate weight ## 0: 47449, 1: 43511
trainDEFSDf = trainDEFSDf[can]
Yy_train = trainDEFSDf["type"]  # 0.9585; 1.0453
sampleW = []
for w in Yy_train:
    if w == 0:
        sampleW.append(0.9585)
    elif w == 1:
        sampleW.append(1.0453) 


In [9]:
## input ext val set ## 6075 x 20 df
extDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\0d1\\df_ext_cbMSIn0d14nonInDI_STDnorm.csv")
#extDEFSDf[extDEFSDf.type .== 1, :]
## calculate weight ## 0: 2943, 1: 3132
extDEFSDf = extDEFSDf[can]
Yy_ext = extDEFSDf["type"]  # 1.0321; 0.9698
sampleExtW = []
for w in Yy_ext:
    if w == 0:
        sampleExtW.append(1.0321)
    elif w == 1:
        sampleExtW.append(0.9698) 


In [10]:
## input ingested set ## 97035 x 20 df
ingestedDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\0d1\\df_ingested_cbMSIn0d14nonInDI_STDnorm.csv")
#ingestedDEFSDf[extDEFSDf.type .== 1, :]
## calculate weight ## 0: 50392, 1: 46643
ingestedDEFSDf = ingestedDEFSDf[can]
Yy_ingested = ingestedDEFSDf["type"]  # 0.9628; 1.0402
sampleIngestedW = []
for w in Yy_ingested:
    if w == 0:
        sampleIngestedW.append(0.9628)
    elif w == 1:
        sampleIngestedW.append(1.0402)


In [11]:
## input FNA set ## 88701 x 20 df
fnaDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\0d1\\df_FNA_cbMSIn0d14nonInDI_STDnorm.csv")
#fnaDEFSDf[fnaDEFSDf.type .== 1, :]
## calculate weight ##  0: 44540, 1: 44161
fnaDEFSDf = fnaDEFSDf[can]
Yy_FNA = fnaDEFSDf["type"]  # 0.9957; 1.0043
sampleFNAW = []
for w in Yy_FNA:
    if w == 0:
        sampleFNAW.append(0.9957)
    elif w == 1:
        sampleFNAW.append(1.0043)


In [12]:
## input DirectIn set ## 88701 x 20 df
diDEFSDf = pd.read_csv(r"I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\0d1\\df_nonInDI_cbMSIn0d14nonInDI_STDnorm.csv")
#diDEFSDf[diDEFSDf.type .== 1, :]
## calculate weight ## 0: 3027, 1: 3030
diDEFSDf = diDEFSDf[dican]

Yy_DI = diDEFSDf["type"]  # 1.0007; 0.9993
sampleDiW = []
for w in Yy_DI:
    if w == 0:
        sampleDiW.append(1.0007)
    elif w == 1:
        sampleDiW.append(0.9993)


In [13]:
diDEFSDf

,pixel_id,311.15,269.25,435.2,301.25,738.55,857.55,867.55,299.1,794.55,233.0,328.15,type
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.351882,-0.007523,-0.275452,-0.018022,0.672710,0.751285,-0.274304,0.104071,0.458268,2.607902,-0.665545,0
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.261211,0.632294,0.224982,0.545462,0.382409,0.957339,-0.888310,0.287721,0.083533,2.722623,-0.675286,0
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.829731,0.873876,0.484333,1.216397,1.746410,0.770691,0.193808,0.549892,1.119869,3.540730,0.469352,0
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.581830,0.884014,0.073722,0.799104,0.841689,0.528149,-0.198573,0.551997,0.406299,2.585243,-0.243567,0
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.287540,0.843223,0.098789,0.978568,0.936361,0.250933,-0.130672,0.086696,0.213206,3.240843,-0.210680,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3023,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,0.079206,0.720608,0.280609,0.366357,0.481351,0.910884,0.318526,0.488964,1.201150,0.042979,0.054728,1
3024,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-1.701625,-0.859929,-0.624510,-0.391325,1.325817,0.672499,0.610001,-0.010228,1.003853,-0.078936,-0.693128,1
3025,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.545081,0.032194,0.801740,-0.594013,0.173474,1.556445,-0.128469,0.235942,-0.068434,-0.236495,-0.773820,1
3026,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.408405,-0.566704,0.217085,-0.262155,0.396137,0.607625,0.134234,-0.363255,1.027061,-0.095828,-0.875789,1


In [14]:
## define functions for performace evaluation ##

# Average score
def avgScore(arrAcc, cv):
    sumAcc = 0
    for acc in arrAcc:
        sumAcc += acc
    return sumAcc / cv


In [15]:
trainDEFSDf[trainDEFSDf.columns[1:-1]]

,311.15,269.25,435.2,301.25,738.55,857.55,867.55,299.1,794.55,233.0,328.15
0,3.953210,0.561634,0.526007,-0.160672,-1.060231,-0.241555,-0.261111,2.281684,-0.476776,0.755008,-0.975763
1,5.499377,1.488873,0.256846,-0.271471,-0.871081,-0.168275,-0.261111,3.225803,-0.476776,1.730237,-0.553603
2,3.655719,0.621905,-0.137142,-0.057255,-1.106408,-0.443515,-0.261111,1.937129,-0.476776,2.006355,-1.057817
3,3.412890,0.428166,-0.086066,-0.275708,-1.058673,-0.913550,-0.261111,1.543656,1.612714,1.552337,-1.187617
4,2.092271,0.750785,0.624481,-0.051827,-1.077604,-0.591638,-0.261111,-0.334251,-0.476776,0.464660,-1.028013
...,...,...,...,...,...,...,...,...,...,...,...
90955,0.273014,-1.042871,1.085864,-0.876582,-1.416813,-1.308037,-0.261111,-0.334251,-0.476776,0.687848,-0.532158
90956,0.096808,-1.042871,1.345307,-1.077784,-0.614252,-1.469057,-0.261111,2.964883,-0.476776,0.410449,-0.691905
90957,0.213884,1.253220,0.874884,-1.425507,-0.478005,-1.083904,-0.261111,2.279959,-0.476776,1.274415,-0.836330
90958,0.402346,-1.042871,0.667639,-1.401248,-0.380958,-0.753560,-0.261111,2.972262,-0.476776,0.654135,-0.776998


In [16]:
import warnings
warnings.filterwarnings('ignore')

# 2. Unsupervised Learning

## 2.1. Dataset Preparation

In [17]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [18]:
df_ROI_for_ML_Opti = pd.concat([trainDEFSDf, extDEFSDf, fnaDEFSDf, diDEFSDf]).set_index('pixel_id')

In [19]:
df_ROI_for_ML_Opti_ingested = ingestedDEFSDf.set_index('pixel_id')

In [20]:
df_ROI_for_ML_Opti_train = trainDEFSDf.set_index('pixel_id')

In [21]:
df_ROI_for_ML_Opti_ext = extDEFSDf.set_index('pixel_id')

In [22]:
df_ROI_for_ML_Opti_FNA = fnaDEFSDf.set_index('pixel_id')

In [23]:
df_ROI_for_ML_Opti_di = diDEFSDf.set_index('pixel_id')

In [24]:
df_ROI_for_ML_Opti_di.describe()

,311.15,269.25,435.2,301.25,738.55,857.55,867.55,299.1,794.55,233.0,328.15,type
count,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3.028000e+03,3028.000000
mean,-2.444837e-16,4.156737e-16,1.795416e-16,-2.902419e-16,2.618172e-16,3.929596e-17,1.056678e-15,-4.223834e-16,-3.808600e-17,-1.976989e-16,6.406148e-16,0.500330
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.500082
min,-2.975432e+00,-3.892500e+00,-2.409224e+00,-4.464815e+00,-2.698140e+00,-2.595695e+00,-2.226096e+00,-4.545126e+00,-3.716094e+00,-2.302102e+00,-4.393321e+00,0.000000
25%,-7.632946e-01,-5.390773e-01,-7.568766e-01,-7.295075e-01,-6.029624e-01,-4.562726e-01,-5.288536e-01,-6.251644e-01,-5.479384e-01,-4.848036e-01,-6.634363e-01,0.000000
50%,6.860680e-02,3.055299e-02,1.295105e-02,-6.668012e-02,-1.605903e-01,4.829007e-02,2.141389e-02,6.114385e-03,1.002588e-02,-2.004024e-01,2.004205e-02,1.000000
75%,7.184720e-01,6.336259e-01,6.577570e-01,7.535412e-01,3.851399e-01,5.778226e-01,5.570340e-01,6.191649e-01,5.574796e-01,9.436113e-02,5.895820e-01,1.000000
max,5.734749e+00,5.911955e+00,3.891428e+00,4.610360e+00,6.890536e+00,4.264909e+00,6.407897e+00,5.484286e+00,5.713480e+00,4.802550e+00,4.083669e+00,1.000000


## 2.2. PCA Analysis

In [37]:
## PCA Analysis

def pca_visual(df=df_ROI_for_ML_Opti_di):

    pca = PCA(n_components=3)
    raMSI_ML_mz_df = df.drop("type", axis = 1)
    pca.fit(raMSI_ML_mz_df)
    result = pd.DataFrame(pca.transform(raMSI_ML_mz_df), columns=['PCA%i' % i for i in range(3)], index=df_ROI_for_ML_Opti_di.index)
    
    plt.subplots(dpi = 300)
    plt.scatter(result['PCA1'], result['PCA2'], c = df['type'], s=1)
    plt.xlabel('PC1', size=20)
    plt.ylabel('PC2', size=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/cbMSIn_0d1/PCA_di.tif", bbox_inches = 'tight')

    components = pca.fit_transform(raMSI_ML_mz_df)
    total_var = pca.explained_variance_ratio_.sum() * 100
    fig = px.scatter_3d(
        components, x=0, y=1, z=2, color=df['type'],
        title = f'Total Explained Variance: {total_var:.2f}%',
        labels = {'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
    )
    fig.show()

In [ ]:
pca_visual()

# 3. Supervised Learning

## 3.1. Preparation of training and test dataset

In [25]:
def split_dataset(df=df_ROI_for_ML_Opti_ingested):
    raMSI_ML_NoType = df.drop("type", axis = 1)
    raMSI_ML_Type=df[['type']]
    X_train,X_test,y_train,y_test=train_test_split(raMSI_ML_NoType,raMSI_ML_Type,test_size=0.2,random_state=42)
    return X_train,X_test,y_train,y_test

X_train,X_test,y_train,y_test=split_dataset(df=df_ROI_for_ML_Opti)

In [26]:
X_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]]

In [27]:
y_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[-1]]

In [28]:
X_train = df_ROI_for_ML_Opti_train[df_ROI_for_ML_Opti_train.columns[:-1]]

In [29]:
y_train = df_ROI_for_ML_Opti_train[df_ROI_for_ML_Opti_train.columns[-1]]

In [30]:
X_val = df_ROI_for_ML_Opti_ext[df_ROI_for_ML_Opti_ext.columns[:-1]]

In [31]:
y_val = df_ROI_for_ML_Opti_ext[df_ROI_for_ML_Opti_ext.columns[-1]]

In [32]:
X_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[:-1]]

In [33]:
y_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[-1]]

In [34]:
X_di = df_ROI_for_ML_Opti_di[df_ROI_for_ML_Opti_di.columns[:-1]]

In [35]:
y_di = df_ROI_for_ML_Opti_di[df_ROI_for_ML_Opti_di.columns[-1]]

## 3.2 Machine Learning Modeling: Model 1+2 (Train 10:10, Val 4:4, Test 3:3)

In [36]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

### SVM

In [37]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_ingested, yVal=y_val, yTest=y_FNA, yTest2=y_di):
    model_svm = LinearSVC(penalty='l2', loss="hinge", C=402, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print("FNA Test Set Precision:", metrics.precision_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    # Test2 set
    predictions_di = model_svm.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Set Precision:", precision_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    
    return x, y

In [38]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.852696054788789
Training Set MCC: 0.7422630358638165
Training Set Accuracy: 0.8692502198768689
              precision    recall  f1-score   support

           0       0.82      0.95      0.88     47449
           1       0.94      0.78      0.85     43511

    accuracy                           0.87     90960
   macro avg       0.88      0.87      0.87     90960
weighted avg       0.88      0.87      0.87     90960

[[45188  2261]
 [ 9632 33879]]
Ext Val Set F1: 0.53737288243983
Ext Val Set MCC: 0.11197101627893552
Ext Val Set Accuracy: 0.5545679012345679
              precision    recall  f1-score   support

           0       0.54      0.60      0.56      2943
           1       0.58      0.52      0.54      3132

    accuracy                           0.55      6075
   macro avg       0.56      0.56      0.55      6075
weighted avg       0.56      0.55      0.55      6075

[[1753 1190]
 [1516 1616]]
5F-CV: 0.7714561623835937
FNA Set F1: 0.7887281279257784
FNA Se

In [39]:
import joblib as jl

In [40]:
model = LinearSVC(penalty='l2', loss="hinge", C=402, random_state=42, class_weight={0:0.9628, 1:1.0402})
model.fit(X_ingested, y_ingested)

LinearSVC(C=402, class_weight={0: 0.9628, 1: 1.0402}, loss='hinge',
          random_state=42)

In [41]:
modelSavePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\modelSVM_cbMSIn0d1.joblib"
jl.dump(model, modelSavePath, compress = 5)

['I:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\raMSIn\\modeling\\modelSVM_cbMSIn0d1.joblib']

## 3.3 Machine Learning Modeling: Model 1 (Train 6:6, Val 4:4, Test 3:3)

### SVM

In [42]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_train, yVal=y_val, yTest=y_FNA, yTest2=y_di):
    model_svm = LinearSVC(penalty='l2', loss="hinge", C=402, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print("FNA Test Set Precision:", metrics.precision_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    # Test2 set
    predictions_di = model_svm.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Set Precision:", precision_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    return x, y

In [43]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.8823608911900969
Training Set MCC: 0.7661635961533722
Training Set Accuracy: 0.883311345646438
              precision    recall  f1-score   support

           0       0.89      0.89      0.89     47449
           1       0.88      0.88      0.88     43511

    accuracy                           0.88     90960
   macro avg       0.88      0.88      0.88     90960
weighted avg       0.88      0.88      0.88     90960

[[42181  5268]
 [ 5346 38165]]
Ext Val Set F1: 0.6189977956181462
Ext Val Set MCC: 0.1862827617304865
Ext Val Set Accuracy: 0.5944032921810699
              precision    recall  f1-score   support

           0       0.59      0.52      0.55      2943
           1       0.60      0.66      0.63      3132

    accuracy                           0.59      6075
   macro avg       0.59      0.59      0.59      6075
weighted avg       0.59      0.59      0.59      6075

[[1536 1407]
 [1057 2075]]
5F-CV: 0.7730393583746793
FNA Set F1: 0.760221669807696
FNA Se

## 3.4 Machine Learning Modeling: Model 2 (Train 4:4, Val 6:6, Test 3:3)

### SVM

In [44]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, XTest2=X_di, yTrain=y_val, yVal=y_train, yTest=y_FNA, yTest2=y_di):
    model_svm = LinearSVC(penalty='l2', loss="hinge", C=402, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(XVal)
    print("Training Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, model_svm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_svm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print("FNA Test Set Precision:", metrics.precision_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    # Test2 set
    predictions_di = model_svm.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_svm.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Set Precision:", precision_score(yTest2, model_svm.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

    
    return x, y

In [45]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.8054057790769563
Training Set MCC: 0.6250436969708872
Training Set Accuracy: 0.8132695690413368
              precision    recall  f1-score   support

           0       0.81      0.84      0.83     47449
           1       0.82      0.78      0.80     43511

    accuracy                           0.81     90960
   macro avg       0.81      0.81      0.81     90960
weighted avg       0.81      0.81      0.81     90960

[[40090  7359]
 [ 9626 33885]]
Ext Val Set F1: 0.8919925450289816
Ext Val Set MCC: 0.7870983339012938
Ext Val Set Accuracy: 0.8930041152263375
              precision    recall  f1-score   support

           0       0.88      0.91      0.89      2943
           1       0.91      0.88      0.89      3132

    accuracy                           0.89      6075
   macro avg       0.89      0.89      0.89      6075
weighted avg       0.89      0.89      0.89      6075

[[2668  275]
 [ 375 2757]]
5F-CV: 0.8894160103123816
FNA Set F1: 0.8272534934314296
FNA 

# 4. Model Evaluation

## 4.1 Prediction

In [46]:
## deploy models for training set ##
model = jl.load(r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\modelSVM_cbMSIn0d1.joblib")
#
## deploy model ##
predicted0n1_train = model.predict(X_train)      
trainDEFSDf["predicted0n1"] = predicted0n1_train
T_ = []
F_ = []
for i in range(len(trainDEFSDf)):
    if trainDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif trainDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
trainDEFSDf["T"] = T_
trainDEFSDf["F"] = F_
#
## save ##, ouputing trainSet df
savePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\df_train_cbMSIn0d1_STDnorm_0n1.csv"
trainDEFSDf.to_csv(savePath)

In [47]:

## deploy model ##
predicted0n1_ext = model.predict(X_val)
extDEFSDf["predicted0n1"] = predicted0n1_ext
T_ = []
F_ = []
for i in range(len(extDEFSDf)):
    if extDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif extDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
extDEFSDf["T"] = T_
extDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\df_ext_cbMSIn0d1_STDnorm_0n1.csv"
extDEFSDf.to_csv(savePath)

In [48]:

## deploy model ##
predicted0n1_FNA = model.predict(X_FNA)
fnaDEFSDf["predicted0n1"] = predicted0n1_FNA
T_ = []
F_ = []
for i in range(len(fnaDEFSDf)):
    if fnaDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif fnaDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
fnaDEFSDf["T"] = T_
fnaDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\df_FNA_cbMSIn0d1_STDnorm_0n1.csv"
fnaDEFSDf.to_csv(savePath)

In [49]:

## deploy model ##
predicted0n1_Di = model.predict(X_di)
diDEFSDf["predicted0n1"] = predicted0n1_Di
T_ = []
F_ = []
for i in range(len(diDEFSDf)):
    if diDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif diDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
diDEFSDf["T"] = T_
diDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"I:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\raMSIn\modeling\df_Di_cbMSIn0d1_STDnorm_0n1.csv"
diDEFSDf.to_csv(savePath)